<a href="https://colab.research.google.com/github/pci-qs/GcodePathGenerator/blob/master/DDos_attack_detection_ensemble_Conv1D%2BConv2D%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
This project is about detection of DDoS attacks on SND (Software defined networks).
The dataset is CIC-IDS2017.

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import json

**Mount the google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**You have to upload CICIDS2017 into your google drive**

Here, check if I can load the dataset files

In [ ]:
foldername='/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive'
for dirname, _, filenames in os.walk(foldername):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV.md5
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/464228_872432_bundle_archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Mornin

# New Section

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

## Join All Datas into 1 DataFrame

In [ ]:
firstData=True
for dirname, _, filenames in os.walk(foldername):
    for filename in filenames:
        if filename[-3:]=='csv':     
            if firstData:
                dataFrame_Total=pd.read_csv(os.path.join(dirname, filename), delimiter=',')
                firstData=False
            else:
                df=pd.read_csv(os.path.join(dirname, filename), delimiter=',')
                dataFrame_Total=pd.concat([dataFrame_Total, df])

## Selection of most significant features

In [ ]:
dataFrame_Total.shape, dataFrame_Total.columns

((2830743, 79),
 Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
        ' Total Backward Packets', 'Total Length of Fwd Packets',
        ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
        ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
        ' Fwd Packet Length Std', 'Bwd Packet Length Max',
        ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
        ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
        ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
        'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
        ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
        ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
        ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
        ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
        ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
        ' Packet Length Std

**Most significant features are selected according to the research paper**

In [ ]:
feature_Selection=[' Bwd Packet Length Std', ' Average Packet Size', ' Flow Duration', ' Flow IAT Std', ' Label']
dataFrame_Selection=dataFrame_Total[feature_Selection]

In [ ]:
dataFrame_Selection.shape

(2830743, 5)

In [ ]:
dataFrame_Benign_DDos=dataFrame_Selection[(dataFrame_Selection[' Label']=='BENIGN') | (dataFrame_Selection[' Label']=='DDoS')]

**Select about 140000 datasets from given dataset**

In [ ]:
rng=np.random.RandomState(10)
nSelectedSamples=140000
nTotalSamples=dataFrame_Benign_DDos.shape[0]
randSamples=rng.random(nTotalSamples)
dataSet_Selected=dataFrame_Benign_DDos[randSamples<nSelectedSamples/nTotalSamples]
dataSet_Selected.shape

(140068, 5)

In [ ]:
dataSet_Selected

,Bwd Packet Length Std,Average Packet Size,Flow Duration,Flow IAT Std,Label
1,0.0,9.000000,109,0.000000,BENIGN
12,0.0,34.000000,4,0.000000,BENIGN
38,0.0,9.000000,1,0.000000,BENIGN
64,0.0,9.000000,1,0.000000,BENIGN
70,0.0,9.000000,32,0.000000,BENIGN
...,...,...,...,...,...
692612,0.0,88.500000,21091,12131.016050,BENIGN
692624,0.0,68.166667,135379,40030.837900,BENIGN
692647,0.0,75.500000,31529,18198.946570,BENIGN
692671,0.0,9.000000,54,0.000000,BENIGN


**Vectorize the column of Label into 1 or 0**

In [ ]:
dataSet_Selected[' Label']=dataSet_Selected[' Label'].where(dataSet_Selected[' Label']=='BENIGN', 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataSet_Selected[' Label']=dataSet_Selected[' Label'].where(dataSet_Selected[' Label']==1, 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataSet_Selected, dataSet_Selected[' Label'].value_counts()

(         Bwd Packet Length Std   Average Packet Size  ...   Flow IAT Std   Label
 1                          0.0              9.000000  ...       0.000000       0
 12                         0.0             34.000000  ...       0.000000       0
 38                         0.0              9.000000  ...       0.000000       0
 64                         0.0              9.000000  ...       0.000000       0
 70                         0.0              9.000000  ...       0.000000       0
 ...                        ...                   ...  ...            ...     ...
 692612                     0.0             88.500000  ...   12131.016050       0
 692624                     0.0             68.166667  ...   40030.837900       0
 692647                     0.0             75.500000  ...   18198.946570       0
 692671                     0.0              9.000000  ...       0.000000       0
 692674                     0.0             88.500000  ...      79.322128       0
 
 [140068 rows 

In [ ]:
dataset_X=dataSet_Selected.iloc[:, :4]
dataset_X

,Bwd Packet Length Std,Average Packet Size,Flow Duration,Flow IAT Std
1,0.0,9.000000,109,0.000000
12,0.0,34.000000,4,0.000000
38,0.0,9.000000,1,0.000000
64,0.0,9.000000,1,0.000000
70,0.0,9.000000,32,0.000000
...,...,...,...,...
692612,0.0,88.500000,21091,12131.016050
692624,0.0,68.166667,135379,40030.837900
692647,0.0,75.500000,31529,18198.946570
692671,0.0,9.000000,54,0.000000


##Scaling of the datasets for input to Model##

Here, the MinMaxScaler of sklearn will be used

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
scaler.fit(dataset_X)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
datasetScaled=scaler.transform(dataset_X)

In [ ]:
# Check if the scaling is done right
iCol=3
datasetScaled[:, iCol].min(), datasetScaled[:, iCol].max(), datasetScaled[:, iCol].std(), 

(0.0, 1.0, 0.07049241575293048)

In [ ]:
X_1d=datasetScaled.reshape(datasetScaled.shape[0], 4,1) # X is just the reshaped dataset for CNN Conv1D input
X_2d=datasetScaled.reshape(datasetScaled.shape[0], 2,2,1) # X is just the reshaped dataset for CNN Conv2D input

# LSTM needs an input data shape larger than 2-dimension.
# datasetScaled is two-dimensional, but every subset of the datasetScaled is one-dimensinoal.
# So we have to reshape subset(4,) into 2-dimensional (2,2,)
X_LSTM=datasetScaled.reshape(datasetScaled.shape[0], 2,2)
y=np.array(dataSet_Selected[' Label']).astype(float) # label for CNN input

##Split dataset into the train & test datasets##

train_test_split of sklearn.model_selection is a good tool to split the datasets

In [40]:
# Split dataset into train datas and test datas
# random_state are set in order to reproduce same splitting
from sklearn.model_selection import train_test_split
X_1d_train, X_1d_test, y_train, y_test=train_test_split(X_1d, y, test_size=0.2, random_state=42)
X_2d_train, X_2d_test, y_train, y_test=train_test_split(X_2d, y, test_size=0.2, random_state=42)
X_LSTM_train, X_LSTM_test, y_train, y_test=train_test_split(X_LSTM, y, test_size=0.2, random_state=42)

##Build the Ensemble CNN model##

In [ ]:
from keras import models
from keras import layers

In [ ]:
nNetsForConv1d=3 # Number of the Conv1D nets for ensembl 
nNetsForConv2d=3 # Number of the Conv2D nets for ensemble
nNetsForLSTM=3   # Number of the LSTM nets for ensemble
nNets= nNetsForConv1d + nNetsForConv2d + nNetsForLSTM

In [ ]:
epochs=30 # Number of epochs for model.fit 
model=[0]*(nNets)
history=[0]* (nNets)

**Models for CONV1D**

please set nNetsForConv1D to set number of conv1d models to ensemble

In [ ]:
kernel_size=3
for i in range(nNetsForConv1d):
    model[i]=models.Sequential()
    model[i].add(layers.Conv1D(32, kernel_size, activation='relu', padding='same', input_shape=(4, 1)))
    model[i].add(layers.Conv1D(64, kernel_size, padding='same', activation='relu'))
    model[i].add(layers.MaxPooling1D(3)) # Removed the MaxPooling2D because the dimension of the data does not fit
    model[i].add(layers.Conv1D(32, 1, padding='same', activation='relu'))
    # model[i].add(layers.MaxPooling2D(1)) # Removed the MaxPooling2D because the dimension of the data does not fit
    model[i].add(layers.Flatten())
    model[i].add(layers.Dense(32, activation='relu'))
    model[i].add(layers.Dense(1, activation='sigmoid'))
    model[i].compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4, 32)             128       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 64)             6208      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 32)             2080      
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

**Models for Conv2D**

In [ ]:
for i in range(nNetsForConv1d, nNetsForConv1d+nNetsForConv2d):
    model[i]=models.Sequential()
    model[i].add(layers.Conv2D(32, (2,2), activation='relu', padding='same', input_shape=(2, 2, 1)))
    model[i].add(layers.Conv2D(64, (2,2), padding='same', activation='relu'))
    model[i].add(layers.MaxPooling2D(2)) # Removed the MaxPooling2D because the dimension of the data does not fit
    model[i].add(layers.Conv2D(32, (1,1), padding='same', activation='relu'))    
    # model[i].add(layers.MaxPooling2D(1)) # Removed the MaxPooling2D because the dimension of the data does not fit
    model[i].add(layers.Flatten())
    model[i].add(layers.Dense(32, activation='relu'))
    model[i].add(layers.Dense(1, activation='sigmoid'))
    model[i].compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model[nNetsForConv1d].summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 2, 2, 32)          160       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 2, 2, 64)          8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 1, 1, 32)          2080      
_________________________________________________________________
flatten_9 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_19 (Dense)             (None, 1)                

**Models for LSTM**

In [ ]:
for i in range(nNetsForConv1d + nNetsForConv2d, nNets):
    model[i]=models.Sequential()
    # input_shape of (2,2) is what we reshaped before
    model[i].add(layers.LSTM(64, input_shape=(2,2))) 
    # activation function is sigmoid because we have a classification problem, filter size=1
    model[i].add(layers.Dense(1, activation='sigmoid')) 
    # optimizer and loss, metrics are usual functions which are most used, most working.
    model[i].compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 

In [ ]:
model[nNetsForConv1d + nNetsForConv2d].summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                17152     
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 65        
Total params: 17,217
Trainable params: 17,217
Non-trainable params: 0
_________________________________________________________________


# Now train the Ensemble CNN model

**Train Conv1D models first**

In [ ]:
for i in range(nNetsForConv1d):
    X_train, X_val, y_train, y_val=train_test_split(X_1d, y, test_size=0.2, random_state=i*10)
    history[i]=model[i].fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
    model_json = model[i].to_json()
    with open("model"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
    model[i].save_weights("modelw"+str(i)+".h5")
    print("Saved model to disk:", i)

Epoch 1/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0990 - acc: 0.9773 - val_loss: 0.0568 - val_acc: 0.9809
Epoch 2/30
876/876 [==============================] - 3s 4ms/step - loss: 0.0477 - acc: 0.9837 - val_loss: 0.0430 - val_acc: 0.9855
Epoch 3/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0430 - acc: 0.9834 - val_loss: 0.0395 - val_acc: 0.9848
Epoch 4/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0371 - acc: 0.9849 - val_loss: 0.1082 - val_acc: 0.9684
Epoch 5/30
876/876 [==============================] - 3s 4ms/step - loss: 0.0318 - acc: 0.9882 - val_loss: 0.0288 - val_acc: 0.9886
Epoch 6/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0293 - acc: 0.9897 - val_loss: 0.0262 - val_acc: 0.9902
Epoch 7/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0261 - acc: 0.9910 - val_loss: 0.0231 - val_acc: 0.9923
Epoch 8/30
876/876 [==============================] - 4s 4ms/step - loss: 0.

**Train Conv2D models**

In [ ]:
for i in range(nNetsForConv1d, nNetsForConv1d + nNetsForConv2d):
    X_train, X_val, y_train, y_val=train_test_split(X_2d, y, test_size=0.2, random_state=i*10)
    history[i]=model[i].fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
    model_json = model[i].to_json()
    with open("model"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
    model[i].save_weights("modelw"+str(i)+".h5")
    print("Saved model to disk:", i)

**Train LSTM models**

In [41]:
for i in range(nNetsForConv1d + nNetsForConv2d, nNets):
    X_train, X_val, y_train, y_val=train_test_split(X_LSTM, y, test_size=0.2, random_state=i*10)
    history[i]=model[i].fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
    model_json = model[i].to_json()
    with open("model"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
        # serialize weights to HDF5
    model[i].save_weights("modelw"+str(i)+".h5")
    print("Saved model to disk:", i)

Epoch 1/30
876/876 [==============================] - 5s 5ms/step - loss: 0.0280 - acc: 0.9893 - val_loss: 0.0267 - val_acc: 0.9888
Epoch 2/30
876/876 [==============================] - 4s 5ms/step - loss: 0.0269 - acc: 0.9893 - val_loss: 0.0255 - val_acc: 0.9898
Epoch 3/30
876/876 [==============================] - 4s 5ms/step - loss: 0.0259 - acc: 0.9896 - val_loss: 0.0243 - val_acc: 0.9900
Epoch 4/30
876/876 [==============================] - 4s 5ms/step - loss: 0.0250 - acc: 0.9898 - val_loss: 0.0237 - val_acc: 0.9904
Epoch 5/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0241 - acc: 0.9899 - val_loss: 0.0223 - val_acc: 0.9905
Epoch 6/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0234 - acc: 0.9901 - val_loss: 0.0214 - val_acc: 0.9905
Epoch 7/30
876/876 [==============================] - 4s 4ms/step - loss: 0.0228 - acc: 0.9902 - val_loss: 0.0218 - val_acc: 0.9911
Epoch 8/30
876/876 [==============================] - 4s 4ms/step - loss: 0.

# Predict with test_data #

In [42]:
predictions=np.zeros((nNets, y_test.shape[0]))

In [43]:
for i in range(nNets):
  if i<nNetsForConv1d:
    predictions[i]=np.ravel(model[i].predict(X_1d_test))
  elif i>=nNetsForConv1d and i< nNetsForConv1d+nNetsForConv2d:
    predictions[i]=np.ravel(model[i].predict(X_2d_test))
  else:
    predictions[i]=np.ravel(model[i].predict(X_LSTM_test))
  



1.   Ensemble with Average voting


For average voting, the float value of predictions will be averaged.
Then if the average of each row is greater than 0.5, it will be malicious (value 1), if not, benign (value 0).


In [44]:
prediction_average_voting=(predictions.mean(axis=0)>0.5).astype(np.int16)

This prediction will be compared with y_test (real label).

In [45]:
acc_average_voting=(y_test==prediction_average_voting).sum()/len(y_test)
acc_average_voting_in_percent=round(acc_average_voting*100, 2)
print(acc_average_voting_in_percent)

99.76




2.   Ensemble with Majority voting



For majority voting, the prediction from CNN models will be set as 0 (benign), and as 1(malicious).
The condition is if the value is greater than 0.5

In [46]:
predictions_int=(predictions>0.5).astype(np.int16)

this values are averaged. if the average of each row is greater than 0.5, the majority will be 1(malicious) because number of malicious prediction will be greater than number of benign predictions for each row.

E.g. a row has prediction of [0,1,0,1,1,0], the average is 0.6666, so 1 is the majority.

In [48]:
prediction_majority_voting=(predictions_int.mean(axis=0)>0.5).astype(np.int16)

In [49]:
acc_majority_voting=(y_test==prediction_majority_voting).sum()/len(y_test)
acc_majority_voting_in_percent=round(acc_majority_voting*100, 2)
print(acc_majority_voting_in_percent)

99.75


# Conclusion

Ensemble of 2 diffenent kind of CNN models, Conv1d, Conv2d delievered very gool prediction, although it has a little calculation time.
With 3 Con1d and 3 Conv2d models, a high accuracy of 99.78% can be reached.
Average voting ensemble delivers a slightly higher accuracy